<a href="https://colab.research.google.com/github/dtuleva/Recipe_Recommendation_System/blob/main/tfrs_quickstart_recreate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mlflow --quiet



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.9 MB/s eta 0:00:00


In [ ]:
!pip install -q tensorflow-recommenders
# !pip install -q --upgrade tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 1.4 MB/s eta 0:00:00


In [ ]:
!zip -r /content/test_experiment.zip /content/mlruns # zip mlflow logs

  adding: content/mlruns/ (stored 0%)
  adding: content/mlruns/0/ (stored 0%)
  adding: content/mlruns/0/38b663884d18471ba617a6ffcbb62e51/ (stored 0%)
  adding: content/mlruns/0/38b663884d18471ba617a6ffcbb62e51/params/ (stored 0%)
  adding: content/mlruns/0/38b663884d18471ba617a6ffcbb62e51/params/dataset (stored 0%)
  adding: content/mlruns/0/38b663884d18471ba617a6ffcbb62e51/meta.yaml (deflated 44%)
  adding: content/mlruns/0/38b663884d18471ba617a6ffcbb62e51/artifacts/ (stored 0%)
  adding: content/mlruns/0/38b663884d18471ba617a6ffcbb62e51/metrics/ (stored 0%)
  adding: content/mlruns/0/38b663884d18471ba617a6ffcbb62e51/tags/ (stored 0%)
  adding: content/mlruns/0/38b663884d18471ba617a6ffcbb62e51/tags/mlflow.source.name (deflated 5%)
  adding: content/mlruns/0/38b663884d18471ba617a6ffcbb62e51/tags/mlflow.runName (stored 0%)
  adding: content/mlruns/0/38b663884d18471ba617a6ffcbb62e51/tags/mlflow.user (stored 0%)
  adding: content/mlruns/0/38b663884d18471ba617a6ffcbb62e51/tags/mlflow.sour

# Recreate tfrs quickstart tutorial



In [ ]:
import mlflow
import pandas as pd

In [ ]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

# import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [ ]:
# experiment = mlflow.create_experiment("tfrs_quickstart_recreate")

In [ ]:
mlflow.autolog()

2024/01/06 10:43:30 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.


##### Recreate dataset structure with recipe data

In [ ]:
ratings_data = pd.read_csv("/content/mock-data_interaction.csv")
recipes_data = pd.read_csv("/content/mock-data_recipe.csv")

In [ ]:
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings_data)) # dict for columns of  different dtypes

In [ ]:
recipes = tf.data.Dataset.from_tensor_slices(recipes_data.recipe_id)

In [ ]:
# feature selection
ratings = ratings.map(lambda x: {
    "recipe_id": x["recipe_id"],
    "user_id": x["user_id"]
})

In [ ]:
for r in ratings.take(1):
  print(type(r))
  for k, v in r.items():
    print(f"{k}: {v}")

<class 'dict'>
recipe_id: 222388
user_id: 8542392


In [ ]:
for r in recipes.take(1):
  print(r)

tf.Tensor(222388, shape=(), dtype=int64)


Build vocabularies to convert user ids and movie titles into integer indices for embedding layers:

In [ ]:
user_id_vocabulary = tf.keras.layers.IntegerLookup(mask_token = None)
user_id_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

recipe_id_vocabulary = tf.keras.layers.IntegerLookup(mask_token = None)
recipe_id_vocabulary.adapt(recipes)

In [ ]:
inverse_user_id_vocabulary = tf.keras.layers.IntegerLookup(vocabulary = user_id_vocabulary.get_vocabulary(), invert = True)
inverse_recipe_id_vocabulary = tf.keras.layers.IntegerLookup(vocabulary = recipe_id_vocabulary.get_vocabulary(), invert = True)

In [ ]:
user_id_vocabulary.vocabulary_size(), inverse_user_id_vocabulary.vocabulary_size()

(8267, 8267)

In [ ]:
recipe_id_vocabulary.vocabulary_size(), inverse_recipe_id_vocabulary.vocabulary_size()

(101, 101)

In [ ]:
user_id_vocabulary(8542392)

<tf.Tensor: shape=(), dtype=int64, numpy=2416>

In [ ]:
inverse_user_id_vocabulary(2416)

<tf.Tensor: shape=(), dtype=int64, numpy=8542392>

In [ ]:
recipe_id_vocabulary(222388)

<tf.Tensor: shape=(), dtype=int64, numpy=32>

In [ ]:
inverse_recipe_id_vocabulary(32)

<tf.Tensor: shape=(), dtype=int64, numpy=222388>

## Define a model
We can define a TFRS model by inheriting from tfrs.Model and implementing the compute_loss method:

In [ ]:
class RecipeModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      recipe_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.recipe_model = recipe_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    recipe_embeddings = self.recipe_model(features["recipe_id"])

    return self.task(user_embeddings, recipe_embeddings)

### Define the two models and the retrieval task.

In [ ]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_id_vocabulary,
    tf.keras.layers.Embedding(user_id_vocabulary.vocabulary_size(), 64)
])
recipe_model = tf.keras.Sequential([
    recipe_id_vocabulary,
    tf.keras.layers.Embedding(recipe_id_vocabulary.vocabulary_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    recipes.batch(100).map(recipe_model)
  )
)


### Fit and evaluate it.

Create the model, train it, and generate predictions:

In [ ]:
test_user_idx = 2416

In [ ]:
# Create a retrieval model.
model = RecipeModel(user_model, recipe_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

In [ ]:
with mlflow.start_run() as run:
  # Train for 3 epochs.
  model.fit(ratings.batch(4096), epochs=3)
  mlflow.log_param("dataset", "mock_100")
  # mlflow.log_metric("factorized_top_k/top_1_categorical_accuracy", model.metrics)


2024/01/06 10:43:49 WARNING mlflow.tensorflow: Encountered unexpected error while inferring batch size from training dataset: The layer "recipe_model_2" has never been called and thus has no defined input shape. Note that the `input_shape` property is only available for Functional and Sequential models.
2024/01/06 10:43:49 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/usr/local/lib/python3.10/dist-packages/mlflow/tensorflow/_autolog.py:52: UserWarning: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data."


Epoch 1/3
3/3 [==============================] - 2s 287ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0029 - factorized_top_k/top_5_categorical_accuracy: 0.0251 - factorized_top_k/top_10_categorical_accuracy: 0.0559 - factorized_top_k/top_50_categorical_accuracy: 0.4915 - factorized_top_k/top_100_categorical_accuracy: 0.9977 - loss: 18512.6555 - regularization_loss: 0.0000e+00 - total_loss: 18512.6555
Epoch 2/3
3/3 [==============================] - 1s 273ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0352 - factorized_top_k/top_5_categorical_accuracy: 0.1389 - factorized_top_k/top_10_categorical_accuracy: 0.2387 - factorized_top_k/top_50_categorical_accuracy: 0.5916 - factorized_top_k/top_100_categorical_accuracy: 0.9953 - loss: 18304.2974 - regularization_loss: 0.0000e+00 - total_loss: 18304.2974
Epoch 3/3
3/3 [==============================] - 1s 277ms/step - factorized_top_k/top_1_categorical_accuracy: 0.5576 - factorized_top_k/top_5_categorical_accuracy: 0.9739

2024/01/06 10:43:54 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: Cannot log input example or model signature for input with type <class 'tensorflow.python.data.ops.batch_op._BatchDataset'>. TensorFlow Keras autologging can only log input examples and model signatures for the following input types: numpy.ndarray, dict[string -> numpy.ndarray], tensorflow.keras.utils.Sequence, and tensorflow.data.Dataset (TensorFlow >= 2.1.0 required)
2024/01/06 10:43:54 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/01/06 10:43:54 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during tensorflow autologging: Model <__main__.RecipeModel object at 0x7d5c56c3d4e0> cannot be saved either because the input shape is not availab

In [ ]:




# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    recipes.batch(100).map(lambda recipe_id: (recipe_id, model.recipe_model(recipe_id))))

# Get some recommendations.
_, recommendations = index(np.array([2416])) # top candidate scores, top candidate identifiers
print(f"Top 3 recommendations for user 2416: {recommendations[0, :3]}")

Top 3 recommendations for user 2416: [ 22402 236920 230270]


In [ ]:
index(np.array([2416]))

(<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
 array([[0.3299513 , 0.2462537 , 0.23303875, 0.23258811, 0.21546024,
         0.2124027 , 0.20341575, 0.2000542 , 0.19347173, 0.19252276]],
       dtype=float32)>,
 <tf.Tensor: shape=(1, 10), dtype=int64, numpy=
 array([[ 22402, 236920, 230270,  26623, 231412,  57002,  14664, 245714,
          17813,  53650]])>)

In [ ]:
test_user_ratings = ratings_data[ratings_data.user_id == inverse_user_id_vocabulary(test_user_idx).numpy()]
test_user_ratings

,Unnamed: 0,user_id,recipe_id,rating,dateLastModified
0,0,8542392,222388,5,2017-04-22T12:46:43.663\n


In [ ]:
rated_recipes = test_user_ratings.recipe_id

In [ ]:
recipes_data[recipes_data.recipe_id.isin(rated_recipes)][["recipe_id", "recipe_name", "review_nums"]]

,recipe_id,recipe_name,review_nums
0,222388,Homemade Bacon,3


In [ ]:
recipes_data[recipes_data.recipe_id.isin(recommendations.numpy().ravel())][["recipe_id", "recipe_name", "review_nums"]]

,recipe_id,recipe_name,review_nums
4,245714,Potato Bacon Pizza,2
8,22402,Cranberry Pork Chops II,100
24,236920,Sweet and Sour Ground Pork Stir-Fry,24
36,26623,Mexican Meatloaf,31
56,231412,Jambalaya In a Jiffy,21
61,53650,Chalau,27
76,57002,Pork Tenderloin with Dijon Marsala Sauce,581
78,230270,Beefy Lasagna Roll-Ups,33
91,17813,Thai Breaded Chicken Filets,17
93,14664,Cilantro and Lime Butter,16


Yo ho ho and a plate of pork